In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# ---------- data ----------
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

X = train.drop(columns=["target"])
y = train["target"]
X_test = test.copy()

# ---------- pipeline ----------
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(
        solver="liblinear",
        max_iter=300,
        tol=1e-3,
        class_weight="balanced"
    ))
])

# ---------- cross-validation ----------
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

scoring = {
    "accuracy": "accuracy",
    "f1": "f1",
    "roc_auc": "roc_auc"
}

cv_results = cross_validate(
    pipe,
    X,
    y,
    cv=cv,
    scoring=scoring,
    n_jobs=-1,
    return_train_score=False
)

# ---------- CV results ----------
for metric in scoring.keys():
    scores = cv_results[f"test_{metric}"]
    print(
        f"{metric}: "
        f"mean = {scores.mean():.4f}, "
        f"std = {scores.std():.4f}, "
        f"folds = {np.round(scores, 4)}"
    )

# ---------- final training ----------
pipe.fit(X, y)

test_pred = pipe.predict(X_test)
test_proba = pipe.predict_proba(X_test)[:, 1]

### база

In [ ]:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=1000))
])

scoring = {
    "accuracy": "accuracy",
    "f1": "f1",
    "roc_auc": "roc_auc"
}

cv_results = cross_validate(
    pipe,
    X_train,
    y_train,
    cv=5,
    scoring=scoring
)

for metric in scoring:
    print(metric, np.mean(cv_results[f"test_{metric}"]))

param_grid = {
    "logreg__C": [0.01, 0.1, 1, 10, 100],
    "logreg__penalty": ["l1", "l2"],
    "logreg__solver": ["liblinear"],
    "logreg__class_weight": [None, "balanced"]
}

gs = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="roc_auc",
    cv=5,
    n_jobs=-1
)

gs.fit(X_train, y_train)

best_model = gs.best_estimator_

y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

### Randomized search cv

In [ ]:
import pandas as pd
from scipy.stats import loguniform

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# ---------- data ----------
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

X = train.drop(columns=["target"])
y = train["target"]
X_test = test.copy()

# ---------- model ----------
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(
        solver="liblinear",
        max_iter=300,   # ⏱ быстрее
        tol=1e-3
    ))
])

cv = StratifiedKFold(
    n_splits=3,        # ⏱ меньше фолдов
    shuffle=True,
    random_state=42
)

param_dist = {
    "logreg__C": loguniform(1e-3, 1e2),   # ⏱ суженный диапазон
    "logreg__penalty": ["l1", "l2"],
    "logreg__class_weight": [None, "balanced"]
}

rs = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=20,          # ⏱ жёсткий лимит
    scoring="roc_auc",
    cv=cv,
    n_jobs=-1,
    random_state=42
)

rs.fit(X, y)

best_model = rs.best_estimator_

test_pred = best_model.predict(X_test)
test_proba = best_model.predict_proba(X_test)[:, 1]

### Bayes search cv

In [9]:
pip install scikit-optimize


   -------------------------- ------------- 2/3 [scikit-optimize]
   ---------------------------------------- 3/3 [scikit-optimize]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from skopt import BayesSearchCV
from skopt.space import Real, Categorical

# ---------- data ----------
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

X = train.drop(columns=["target"])
y = train["target"]
X_test = test.copy()

# ---------- model ----------
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(
        solver="liblinear",
        max_iter=300,   # ⏱
        tol=1e-3
    ))
])

cv = StratifiedKFold(
    n_splits=3,        # ⏱
    shuffle=True,
    random_state=42
)

search_space = {
    "logreg__C": Real(1e-3, 1e2, prior="log-uniform"),
    "logreg__penalty": Categorical(["l1", "l2"]),
    "logreg__class_weight": Categorical([None, "balanced"])
}

opt = BayesSearchCV(
    pipe,
    search_space,
    n_iter=25,         # ⏱ лимит итераций
    scoring="roc_auc",
    cv=cv,
    n_jobs=-1,
    random_state=42
)

opt.fit(X, y)

best_model = opt.best_estimator_

test_pred = best_model.predict(X_test)
test_proba = best_model.predict_proba(X_test)[:, 1]

### Optuna

In [11]:
pip install optuna

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 840.2 kB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.1 MB 860.9 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 898.8 kB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 898.8 kB/s eta 0:00:02
   ------------------------ --------------- 1.3/2.1 MB 838.9 kB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 838.9 kB/s eta 0:00:01
   ------------------------ --------------- 1.3/2.1 MB 838.9 kB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 742.6 kB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 768.4 kB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 768.4 kB/s eta 0:00:01
   ----------

In [ ]:
import pandas as pd
import optuna

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# ---------- data ----------
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

X = train.drop(columns=["target"])
y = train["target"]
X_test = test.copy()

cv = StratifiedKFold(
    n_splits=3,        # ⏱
    shuffle=True,
    random_state=42
)

def objective(trial):
    model = Pipeline([
        ("scaler", StandardScaler()),
        ("logreg", LogisticRegression(
            C=trial.suggest_float("C", 1e-3, 1e2, log=True),
            penalty=trial.suggest_categorical("penalty", ["l1", "l2"]),
            class_weight=trial.suggest_categorical("class_weight", [None, "balanced"]),
            solver="liblinear",
            max_iter=300,   # ⏱
            tol=1e-3
        ))
    ])

    scores = cross_val_score(
        model,
        X,
        y,
        cv=cv,
        scoring="roc_auc",
        n_jobs=-1
    )

    return scores.mean()

study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.MedianPruner()  # ⏱ ранняя остановка
)

study.optimize(
    objective,
    timeout=300        # ⏱ 5 минут ЖЁСТКО
)

best_model = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(
        **study.best_params,
        solver="liblinear",
        max_iter=300,
        tol=1e-3
    ))
])

best_model.fit(X, y)

test_pred = best_model.predict(X_test)
test_proba = best_model.predict_proba(X_test)[:, 1]